In [1]:
!pip install torch
!pip install transformers
!pip install datasets  # Required for BLEU, Rouge-L, BERTScore
!pip install bert-score
!pip install codebleu  # Required for CodeBLEU


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-m

In [2]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load GPT-2 and its tokenizer
model_name = 'gpt2'  #
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Function to get model outputs with hidden states
def get_model_outputs(text, model, tokenizer):
    inputs = tokenizer(text, return_tensors='pt')
    with torch.no_grad():
        outputs = model(**inputs, output_hidden_states=True)
    return outputs


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [3]:
def generate_text_from_layer(outputs, tokenizer, layer_num):
    # Get the hidden state for the specified layer
    hidden_state = outputs.hidden_states[layer_num]

    # Project the hidden state to vocabulary size and apply softmax to get probabilities
    logits = model.transformer.lm_head(hidden_state)
    probabilities = torch.softmax(logits, dim=-1)

    # Get the token with the highest probability
    predicted_token_ids = torch.argmax(probabilities, dim=-1)

    # Decode the token IDs to text
    predicted_text = tokenizer.batch_decode(predicted_token_ids, skip_special_tokens=True)

    return predicted_text


In [8]:
!pip install rouge_score


from datasets import load_metric

# Load metrics
bleu_metric = load_metric("bleu")
rouge_metric = load_metric("rouge")
bert_score = load_metric("bertscore")
# code_bleu = load_metric("codebleu")

# Function to evaluate text with different metrics
def evaluate_text_metrics(reference, hypothesis):
    # Prepare references and hypothesis
    references = [[reference]]
    hypotheses = [hypothesis]

    # Calculate BLEU
    bleu_result = bleu_metric.compute(predictions=hypotheses, references=references)

    # Calculate Rouge-L
    rouge_result = rouge_metric.compute(predictions=hypotheses, references=references)

    # Calculate BERTScore
    bert_result = bert_score.compute(predictions=hypotheses, references=references)

    # Calculate CodeBLEU
    code_bleu_result = code_bleu.compute(predictions=hypotheses,)

    return {
        "BLEU": bleu_result,
        "Rouge-L": rouge_result,
        "BERTScore": bert_result,
        "CodeBLEU": code_bleu_result,
    }


/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for bleu contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/bleu/bleu.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major r

In [12]:
def generate_text_from_layer(outputs, tokenizer, layer_num):
    # Get the hidden state for the specified layer
    hidden_state = outputs.hidden_states[layer_num]

    # Project the hidden state to vocabulary size and apply softmax to get probabilities
    logits = model.lm_head(hidden_state)  # This line is causing the error

    # Apply softmax to get probabilities
    probabilities = torch.softmax(logits, dim=-1)

    # Get the token with the highest probability
    predicted_token_ids = torch.argmax(probabilities, dim=-1)

    # Decode the token IDs to text
    predicted_text = tokenizer.batch_decode(predicted_token_ids, skip_special_tokens=True)

    return predicted_text
from transformers import GPT2LMHeadModel, GPT2Tokenizer

model_name = 'gpt2'  # Ensure this is the correct model name
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)  # Ensure using LMHead model
def generate_text_from_layer(outputs, tokenizer, layer_num):
    # Get the hidden state for the specified layer
    hidden_state = outputs.hidden_states[layer_num]

    # Project the hidden state to the vocabulary size using the correct method
    logits = model.transformer.lm_head(hidden_state)  # Accessing the language modeling head

    # Apply softmax to get probabilities
    probabilities = torch.softmax(logits, dim=-1)

    # Get the token with the highest probability
    predicted_token_ids = torch.argmax(probabilities, dim=-1)

    # Decode the token IDs to text
    predicted_text = tokenizer.batch_decode(predicted_token_ids, skip_special_tokens=True)

    return predicted_text


In [16]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

model_name = 'gpt2'  # Ensure this is the correct model name
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)  # Correct model class for text generation
def generate_text_from_layer(outputs, tokenizer, layer_num):
    # Get the hidden state for the specified layer
    hidden_state = outputs.hidden_states[layer_num]

    # Project the hidden state to the vocabulary size using the language modeling head
    logits = model.lm_head(hidden_state)  # Accessing the correct "lm_head"

    # Apply softmax to get probabilities
    probabilities = torch.softmax(logits, dim=-1)

    # Get the token with the highest probability
    predicted_token_ids = torch.argmax(probabilities, dim=-1)

    # Decode the token IDs to text
    predicted_text = tokenizer.batch_decode(predicted_token_ids, skip_special_tokens=True)

    return predicted_text


In [18]:
# Function to evaluate text with different metrics
def evaluate_text_metrics(references, hypotheses):
    # Ensure references and hypotheses are lists of lists
    if isinstance(references, str):
        references = [[references]]  # Wrap the string in two levels of lists
    if isinstance(hypotheses, str):
        hypotheses = [hypotheses]  # Wrap the string in a list

    # Calculate BLEU
    bleu_result = bleu_metric.compute(predictions=hypotheses, references=references)

    # Calculate Rouge-L
    rouge_result = rouge_metric.compute(predictions=hypotheses, references=references)

    # Calculate BERTScore
    bert_result = bert_score.compute(predictions=hypotheses, references=references)

    return {
        "BLEU": bleu_result,
        "Rouge-L": rouge_result,
        "BERTScore": bert_result,
    }


In [20]:
# Reference text should be a list of lists
reference_text = [["The quick brown fox jumps over the lazy dog."]]  # Double list for references

# Example generated text should be a list
generated_text = ["The quick brown fox jumps over the lazy cat."]  # List for predictions

# Function to evaluate text with different metrics
def evaluate_text_metrics(references, predictions):
    # Ensure references and predictions are in the correct list format
    if isinstance(references, str):
        references = [[references]]  # Wrap the string in two levels of lists
    if isinstance(predictions, str):
        predictions = [predictions]  # Wrap the string in a list

    # Calculate BLEU
    bleu_result = bleu_metric.compute(predictions=predictions, references=references)

    # Calculate Rouge-L
    rouge_result = rouge_metric.compute(predictions=predictions, references=references)

    # Calculate BERTScore
    bert_result = bert_score.compute(predictions=predictions, references=references)

    return {
        "BLEU": bleu_result,
        "Rouge-L": rouge_result,
        "BERTScore": bert_result,
    }
reference_text = [["The quick brown fox jumps over the lazy dog."]]  # Double list for references
generated_text = ["The quick brown fox jumps over the lazy cat."]  # List for predictions
metrics = evaluate_text_metrics(reference_text, generated_text)  # Test with corrected format

print("BLEU:", metrics["BLEU"])
print("Rouge-L:", metrics["Rouge-L"])
print("BERTScore:", metrics["BERTScore"])


ValueError: Got a string but expected a list instead: 'The quick brown fox jumps over the lazy cat.'

In [ ]:
# Reference text should be a list of lists
reference_text = [["The quick brown fox jumps over the lazy dog."]]  # Corrected format

# Example generated text (output from the model)
generated_text = ["The quick brown fox jumps over the lazy cat."]  # Correct format

# Evaluate text using different metrics
metrics = evaluate_text_metrics(reference_text, generated_text)  # Corrected to list of lists


In [ ]:
# Define a reference text (the expected output for comparison)
reference_text = "The quick brown fox jumps over the lazy dog."

# Get model outputs
outputs = get_model_outputs(reference_text, model, tokenizer)

# Iterate through specified layers and evaluate text metrics
for layer_num in range(8, 33):  # From layer 8 to 32
    # Generate text from the current layer
    generated_text = generate_text_from_layer(outputs, tokenizer, layer_num)

    # Evaluate text using different metrics
    metrics = evaluate_text_metrics(reference_text, generated_text)

    print(f"Results for Layer {layer_num}:")
    print("BLEU:", metrics["BLEU"])
    print("Rouge-L:", metrics["Rouge-L"])
    print("BERTScore:", metrics["BERTScore"])
    print("CodeBLEU:", metrics["CodeBLEU"])
    print("-" * 30)
